# **QUESTION 1**

In [1]:
################################### Question 1 (Preparation and Data Import) ###################################

import pandas as pd
import numpy as np
import statsmodels.api as sm

df1 = pd.read_csv("OTC_Sales.csv",sep="\t")
df2 = pd.read_csv("OTC_Demographics.csv",sep="\t")
df3 = pd.read_csv("OTC_Instruments.csv",sep="\t")


df1.head()


,store,week,brand,sales_,count,price_,prom_,cost_
0,2,1,1,16,14181,3.29,0.0,2.06
1,2,2,1,12,13965,3.27,0.0,2.04
2,2,3,1,6,13538,3.37,0.0,2.15
3,2,4,1,12,13735,3.30,0.0,2.07
4,2,5,1,10,13735,3.34,0.0,2.12


In [ ]:
################################### Summary statistics for the table 1 ###################################

# Calculate the total sales
total_sales = df1['sales_'].sum()

# Calculate the sales share for each brand
brand_sales = df1.groupby('brand')['sales_'].sum()


# Generate share variables for each brand
share_1 = brand_sales[1] / total_sales
share_2 = brand_sales[2] / total_sales
share_3 = brand_sales[3] / total_sales
share_4 = brand_sales[4] / total_sales
share_5 = brand_sales[5] / total_sales
share_6 = brand_sales[6] / total_sales
share_7 = brand_sales[7] / total_sales
share_8 = brand_sales[8] / total_sales
share_9 = brand_sales[9] / total_sales
share_10 = brand_sales[10] / total_sales
share_11 = brand_sales[11] / total_sales

# Print the share variables
print(f"share_1: {share_1}")
print(f"share_2: {share_2}")
print(f"share_3: {share_3}")
print(f"share_4: {share_4}")
print(f"share_5: {share_5}")
print(f"share_6: {share_6}")
print(f"share_7: {share_7}")
print(f"share_8: {share_8}")
print(f"share_9: {share_9}")
print(f"share_10: {share_10}")
print(f"share_11: {share_11}")


share_1: 0.14513692602512893
share_2: 0.17988041419128795
share_3: 0.11786960705561754
share_4: 0.11845629580078901
share_5: 0.07822151064127274
share_6: 0.035480961401911947
share_7: 0.04076390164464756
share_8: 0.03445288532976568
share_9: 0.0800747891073284
share_10: 0.09615938194808078
share_11: 0.07350332685416947


In [3]:
######################################### Weighted average price and cost for each brand #########################################

for i in range(1, 12):
    weighted_avg_price_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['price_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_price_{i}: {weighted_avg_price_i}")

# Calculate the weighted average cost (wholesale price) for each brand

for i in range(1, 12):
    weighted_avg_cost_i = (df1[df1['brand'] == i]['sales_'] * df1[df1['brand'] == i]['cost_']).sum() / df1[df1['brand'] == i]['sales_'].sum()
    print(f"weighted_avg_cost_{i}: {weighted_avg_cost_i}")


weighted_avg_price_1: 3.414884775217227
weighted_avg_price_2: 4.888457470318381
weighted_avg_price_3: 6.935243987533144
weighted_avg_price_4: 2.938778235511942
weighted_avg_price_5: 5.02520433197813
weighted_avg_price_6: 8.071122701282645
weighted_avg_price_7: 2.670661107001143
weighted_avg_price_8: 3.600319089679319
weighted_avg_price_9: 3.9389793892084364
weighted_avg_price_10: 1.8611150392017106
weighted_avg_price_11: 4.29954384394465
weighted_avg_cost_1: 2.17952871174915
weighted_avg_cost_2: 3.6696483928489783
weighted_avg_cost_3: 5.748667255896171
weighted_avg_cost_4: 2.0292830031475653
weighted_avg_cost_5: 3.6047360857984017
weighted_avg_cost_6: 6.104660794313088
weighted_avg_cost_7: 1.8421487658887619
weighted_avg_cost_8: 2.484719503461447
weighted_avg_cost_9: 3.7093806491372225
weighted_avg_cost_10: 0.9077034925160371
weighted_avg_cost_11: 1.8716247062772742


# **QUESTION 2**


In [4]:
####################################### Question 2 (Data Preparation) #######################################

df1['sales_per_count'] = df1['sales_'] / df1['count']

df1['share_0'] = 1 - df1.groupby(['store', 'week'])['sales_per_count'].transform('sum')

df1['Y'] = np.log(df1['sales_per_count']) - np.log(df1['share_0'])

df1.head()

,store,week,brand,sales_,count,price_,prom_,cost_,sales_per_count,share_0,Y
0,2,1,1,16,14181,3.29,0.0,2.06,0.001128,0.993724,-6.780774
1,2,2,1,12,13965,3.27,0.0,2.04,0.000859,0.993913,-7.053298
2,2,3,1,6,13538,3.37,0.0,2.15,0.000443,0.995199,-7.716683
3,2,4,1,12,13735,3.30,0.0,2.07,0.000874,0.993957,-7.036735
4,2,5,1,10,13735,3.34,0.0,2.12,0.000728,0.995268,-7.220374


In [5]:
##################################### OLS Q2.(a) #####################################

# Define the independent variables
X = df1[['prom_', 'price_']]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model = sm.OLS(Y, X).fit()

# Print the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     315.2
Date:                Fri, 07 Feb 2025   Prob (F-statistic):          1.72e-136
Time:                        01:22:19   Log-Likelihood:                -50711.
No. Observations:               38544   AIC:                         1.014e+05
Df Residuals:                   38541   BIC:                         1.015e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.7660      0.012   -627.345      0.0

In [6]:
######################################## Generate Dummies for Product Type ########################################

# Create dummy variables
df1['tyl'] = df1['brand'].isin([1, 2, 3]).astype(int)
df1['adv'] = df1['brand'].isin([4, 5, 6]).astype(int)
df1['bay'] = df1['brand'].isin([7, 8, 9]).astype(int)
df1['sto'] = df1['brand'].isin([10, 11]).astype(int)

# Display the first few rows to verify
df1.head()

,store,week,brand,sales_,count,price_,prom_,cost_,sales_per_count,share_0,Y,tyl,adv,bay,sto
0,2,1,1,16,14181,3.29,0.0,2.06,0.001128,0.993724,-6.780774,1,0,0,0
1,2,2,1,12,13965,3.27,0.0,2.04,0.000859,0.993913,-7.053298,1,0,0,0
2,2,3,1,6,13538,3.37,0.0,2.15,0.000443,0.995199,-7.716683,1,0,0,0
3,2,4,1,12,13735,3.30,0.0,2.07,0.000874,0.993957,-7.036735,1,0,0,0
4,2,5,1,10,13735,3.34,0.0,2.12,0.000728,0.995268,-7.220374,1,0,0,0


In [7]:
##################################### OLS Q2.(b) #####################################

# Define the independent variables including the dummy variables
X = df1[['price_', 'prom_', 'tyl', 'adv', 'bay', 'sto']]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
model_with_dummies = sm.OLS(Y, X).fit()

# Print the regression results
print(model_with_dummies.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     4535.
Date:                Fri, 07 Feb 2025   Prob (F-statistic):               0.00
Time:                        01:22:19   Log-Likelihood:                -42106.
No. Observations:               38544   AIC:                         8.422e+04
Df Residuals:                   38538   BIC:                         8.427e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.7781      0.009   -648.960      0.0

In [8]:
############################################### Fixed Effects for Everything ###############################################

# Generate new variables for each unique combination of store number and tyl, adv, bay, and sto
for store in df1['store'].unique():
    df1[f'tyl_store_{store}'] = df1['tyl'] * (df1['store'] == store).astype(int)
    df1[f'adv_store_{store}'] = df1['adv'] * (df1['store'] == store).astype(int)
    df1[f'bay_store_{store}'] = df1['bay'] * (df1['store'] == store).astype(int)
    df1[f'sto_store_{store}'] = df1['sto'] * (df1['store'] == store).astype(int)

# Display the first few rows to verify
df1.head()


/tmp/ipykernel_2954/3716583663.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'sto_store_{store}'] = df1['sto'] * (df1['store'] == store).astype(int)
/tmp/ipykernel_2954/3716583663.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'tyl_store_{store}'] = df1['tyl'] * (df1['store'] == store).astype(int)
/tmp/ipykernel_2954/3716583663.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

,store,week,brand,sales_,count,price_,prom_,cost_,sales_per_count,share_0,...,bay_store_121,sto_store_121,tyl_store_122,adv_store_122,bay_store_122,sto_store_122,tyl_store_123,adv_store_123,bay_store_123,sto_store_123
0,2,1,1,16,14181,3.29,0.0,2.06,0.001128,0.993724,...,0,0,0,0,0,0,0,0,0,0
1,2,2,1,12,13965,3.27,0.0,2.04,0.000859,0.993913,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1,6,13538,3.37,0.0,2.15,0.000443,0.995199,...,0,0,0,0,0,0,0,0,0,0
3,2,4,1,12,13735,3.30,0.0,2.07,0.000874,0.993957,...,0,0,0,0,0,0,0,0,0,0
4,2,5,1,10,13735,3.34,0.0,2.12,0.000728,0.995268,...,0,0,0,0,0,0,0,0,0,0


In [9]:
##################################### OLS Q2.(c) Fixed Effects Model #####################################

# Define the independent variables including the dummy variables
X_fixed_effects = df1[['price_', 'prom_'] + [col for col in df1.columns if '_store' in col]]

# Add a constant to the independent variables
X_fixed_effects = sm.add_constant(X_fixed_effects)

# Define the dependent variable
Y = df1['Y']

# Fit the regression model
fixed_effects_model = sm.OLS(Y, X_fixed_effects).fit()

# Print the coefficients for price and promotion
print(fixed_effects_model.params)

const           -7.200177
price_          -0.188035
prom_            0.443860
tyl_store_2      0.824262
adv_store_2      0.155209
                   ...   
sto_store_122    0.141384
tyl_store_123    0.938088
adv_store_123    0.025048
bay_store_123   -0.829285
sto_store_123   -0.217803
Length: 295, dtype: float64


In [13]:
####################################### OLS Q2.(d) #####################################


from statsmodels.sandbox.regression.gmm import IV2SLS

###################################### Q2.(d) #####################################

# Define the instrument variable
instrument = df1['cost_']

# First regression: original model
X = df1[['prom_', 'price_', 'cost_']]
X = sm.add_constant(X)
Y = df1['Y']
model_iv = sm.OLS(Y, X).fit()
print(model_iv.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     866.3
Date:                Fri, 07 Feb 2025   Prob (F-statistic):               0.00
Time:                        01:23:56   Log-Likelihood:                -49766.
No. Observations:               38544   AIC:                         9.954e+04
Df Residuals:                   38540   BIC:                         9.957e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.5713      0.013   -588.577      0.0

In [14]:
# Second regression: model with dummies
X_dummies = df1[['price_', 'prom_', 'tyl', 'adv', 'bay', 'sto','cost_']]
X_dummies = sm.add_constant(X_dummies)
model_iv_dummies = sm.OLS(Y, X_dummies).fit()
print(model_iv_dummies.summary())



                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     4595.
Date:                Fri, 07 Feb 2025   Prob (F-statistic):               0.00
Time:                        01:24:38   Log-Likelihood:                -40623.
No. Observations:               38544   AIC:                         8.126e+04
Df Residuals:                   38537   BIC:                         8.132e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.6167      0.009   -620.789      0.0

In [15]:
# Third regression: fixed effects model
X_fixed_effects = df1[['price_', 'prom_','cost_'] + [col for col in df1.columns if '_store' in col]]
X_fixed_effects = sm.add_constant(X_fixed_effects)
model_iv_fixed_effects = sm.OLS(Y, X_fixed_effects).fit()
print(model_iv_fixed_effects.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.489
Method:                 Least Squares   F-statistic:                     126.5
Date:                Fri, 07 Feb 2025   Prob (F-statistic):               0.00
Time:                        01:25:12   Log-Likelihood:                -37933.
No. Observations:               38544   AIC:                         7.646e+04
Df Residuals:                   38249   BIC:                         7.898e+04
Df Model:                         294                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -6.9975      0.011   -661.028